## Injection benchmark

CPython 3.7.2 and an i7 7700K were used for the timings

In [1]:
import sys
from antidote import __version__
print(f"Antidote: {__version__}")
print(f"Python {sys.version}")

Antidote: 0.3.1.dev40+g4bc744b.d20190203
Python 3.7.2 (default, Dec 29 2018, 19:09:49) 
[GCC 7.3.0]


In [2]:
from antidote import world, register, inject

In [3]:
@register
class Service1:
    pass


@register
class Service2:
    def __init__(self, service1: Service1):
        self.service1 = service1
       
 
@register
class Service3:
    def __init__(self, service1: Service1, service2: Service2):
        self.service1 = service1
        self.service2 = service2

  
@register
class Service4:
    def __init__(self, service1: Service1, service2: Service2, service3: Service3):
        self.service1 = service1
        self.service2 = service2
        self.service3 = service3


### Function call

Injection overhead is here measured with a function which does nothing.

In [4]:
def f(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    return s1, s2, s3, s4

Time necessary to only execute the function, without retrieving the services

In [5]:
args = (world.get(Service1), world.get(Service2), world.get(Service3), world.get(Service4))
%timeit f(*args)

243 ns ± 1.59 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Overhead of the injection when all argument must be retrieved from the container.

In [6]:
f_injected = inject(f)
assert f(*args) == f_injected()
%timeit f_injected()

857 ns ± 16 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Overhead of the injection when no argument has to be retrieved.

In [7]:
assert f(*args) == f_injected(*args)
%timeit f_injected(*args)

331 ns ± 1.74 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


It should be noted that this is the worst scenario possible. In a real case example, the function would be much slower.
To put those results into perspective, the overhead is roughly the time needed to decode this simple JSON.

In [8]:
import json
%timeit json.loads('{ "name":"John", "age":30, "city":"New York"}')

4.34 µs ± 78.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Object instantiation

A similar benchmark is done with object instantiation.

In [9]:
class Obj:
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit Obj(*args)

689 ns ± 1.42 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
class ObjInjected:
    @inject
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit ObjInjected()

1.83 µs ± 35.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Conclusion

The key take away from those benchmarks, is to avoid using injection on short functions which are called repeatedly, in a loop typically. In the other cases, the overhead is considered to be negligeable.